In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
GENIUS_API_TOKEN = '-OaaGmLFE8wuWpY1nEKnDAjF-VJOytR9bDsn5AJEru76Q6RD4TuvZ0GBTCF_VVY_'

In [ ]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

import time
import json

In [ ]:
#limit execution time of a fuction
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [ ]:
# for user input
def request_artist_song_url(artist_name, song_name):
    artist_name = artist_name.lower()
    song_name = song_name.lower()
    artist = artist_name.replace(' ','-')
    artist = artist.capitalize()
    song = song_name.replace(' ','-')
    base_url = 'https://genius.com'
    make_url = base_url + '/' + artist + '-' + song + '-' + 'lyrics'
    return make_url

In [ ]:
request_artist_song_url('One Direction','Perfect')

In [ ]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    response = None
    json = None
    while True:
            response = request_artist_info(artist_name, page) 
            json = response.json()
#             print(json_var)
            # Collect up to song_cap song objects from artist
            song_info = []
#             title_info = []
            song_title_tup=[]
            for hit in json['response']['hits']:
                if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                    song_info.append(hit)
                
            # Collect song URL's from song objects
            for song in song_info:
                if (len(songs) < song_cap):
                    url = song['result']['url']
                    songs.append(url)
                    title = song['result']['title']
                    song_title_tup.append((artist_name,url,title))
            if (len(songs) == song_cap):
                break
            else:
                page += 1
 
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return song_title_tup

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../input/spotifyclassification/data.csv')

In [ ]:
artist_name = df['artist'].unique()
artist_name = artist_name.astype('str')
artist_name = np.char.lower(artist_name)

artist_name

In [ ]:
song_df = pd.DataFrame(columns=["Artist", "Lyrics_url","Song"])

In [ ]:
for artist in artist_name:
    try:
        with time_limit(7):
            new_tup = request_song_url(artist, 3)
            df = pd.DataFrame(new_tup, columns=["Artist", "Lyrics_url","Song"])
            song_df = pd.concat([df,song_df])
    except TimeoutException as e:
        print("{} not found!".format(artist))
        songs_urls.append('NaN')
#     print(len(songs_urls))

In [ ]:
artist='kanye west'
new_tup = request_song_url(artist, 3)

In [ ]:
df = pd.DataFrame(new_tup, columns=["Artist", "Lyrics_url","Song"])
df

In [ ]:
song_df

In [ ]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    lis=''
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
#     print(html)
    lyrics = html.find_all('div', class_='Lyrics__Container-sc-1ynbvzw-10 cvsIWi')
    for ly in lyrics:
        lis += ly.get_text(separator = " ").strip()
    #remove identifiers like chorus, verse, etc
    lis = re.sub(r'[\(\[].*?[\)\]]', '', lis)
    #remove empty lines
    lis = os.linesep.join([s for s in lis.splitlines() if s])         
    return lis



In [ ]:
songs_urls

In [ ]:
# DEMO
print(scrape_song_lyrics(song_urls[2]))